In [1]:
import illustris_python as il
import numpy as np
import h5py
from groupcat import gcPath, offsetPath

In [2]:
from importlib import reload  

In [3]:
from findAccTags import loadIDsSubhalos, loadIDsGroups, getMergerTrees, create_tags

In [4]:
basePath = '/home/tnguser/sims.TNG/TNG100-1/output'
dataPath = '../Particles-TNG-100/'
snapNum0 = 99         
h0= 0.6774

## Find a List of MW-mass haloes at redshift z=0

In [5]:
GroupFirstSub = il.groupcat.loadHalos(basePath,snapNum0,fields=['GroupFirstSub'])

fields = ['SubhaloMassType',]
subhalos = il.groupcat.loadSubhalos(basePath,snapNum0,fields=fields)
dm=subhalos[:,1]*(1e10/h0)
sm=subhalos[:,4]*(1e10/h0)
mask=((dm>0.8e12) & (dm<2e12)) & ((sm>2e10) & (sm<1e11))
arg_list_prospective=np.where(mask)[0]

arg=np.in1d(arg_list_prospective,GroupFirstSub)
subfindID_prospective=arg_list_prospective[arg]

In [6]:
subfindID0=subfindID_prospective[1]

## Find Tags of a Subhalo given by its SubfindID

In [7]:
# Find particle IDs of subhalo at z=0
snapNum0 = 99 
partType = 4                 # stars
particleIDs=il.snapshot.loadSubhalo(basePath, snapNum0, subfindID0, partType, fields=['ParticleIDs',])

In [8]:
# get masks identifying where the particle was born.
(particleIDs_mpb, particleIDs_fof, particleIDs_ext)=create_tags(particleIDs, subfindID0, snapNum0, basePath, dataPath)

In [32]:
print(len(np.where(particleIDs_mpb)[0]))
print(len(np.where(particleIDs_fof)[0]))
print(len(np.where(particleIDs_ext)[0]))

25949
1357
1366
